In [590]:
import numpy as np
import pandas as pd
import powerplantmatching as pm
import datetime as dt

In [93]:
def generate_outage_matrix(seed=42):
    np.random.seed(seed)

    hours_per_year = 365 * 24
    generators = ["gas", "coal", "wind"]
    outage_rates = {"gas": 100, "coal": 200, "wind": 400}
    outage_durations = {"gas": 2, "coal": 3, "wind": 4}

    matrix = np.zeros((len(generators), hours_per_year), dtype=bool)

    for i, gen in enumerate(generators):
        rate = outage_rates[gen]
        duration = outage_durations[gen]
        
        # Generate inter-arrival times (in hours) using exponential distribution
        mean_interval = hours_per_year / rate
        times = np.cumsum(np.random.exponential(mean_interval, rate * 2))
        
        # Only keep outage start times within the year
        outage_starts = times[times < hours_per_year].astype(int)
        
        for start in outage_starts:
            end = min(start + duration, hours_per_year)
            matrix[i, start:end] = True

    return matrix, generators


## Load forced outage statistics

In [591]:
#file2 = snakemake.params.data_folder + "Common data/Common Data.xlsx"
file = "/trinity/home/fuhrand/ERAA/open_eraa/data/Common data/Common Data.xlsx"

excel_file = pd.ExcelFile(file)

properties_raw_eff = pd.read_excel(excel_file, "Common Data", index_col=[2,3], skiprows=10, header=0).dropna(how="all").iloc[2:, 1:].dropna(how="all", axis=1).iloc[:27, 1:17]
properties_raw_outstats = pd.read_excel(excel_file, "Common Data", index_col=[2,3], skiprows=44, header=[0,3]).iloc[:, 1:].dropna(how="all", axis=1).dropna(how="all").iloc[:27, 1:17]

In [592]:
# Check uniqueness of index
if not properties_raw_eff.index.unique().shape[0] == properties_raw_eff.index.shape[0]: raise Exception( "Index of generator type is not unique" )
if not properties_raw_outstats.index.unique().shape[0] == properties_raw_outstats.index.shape[0]: raise Exception( "Index of generator type is not unique" )

In [593]:
# Remove irrelevant columns, rename columns and flag relevant entries
properties_raw_outstats.columns = properties_raw_outstats.columns.droplevel()
properties_raw_outstats.index = properties_raw_outstats.index.set_names(['Fuel', 'Type'])
properties_raw_outstats.rename(columns={'%': 'rel_occ'}, inplace=True)
properties_raw_eff.rename(columns={'Efficiency range in NCV terms': 'eff'}, inplace=True)
properties_stage_outstats = properties_raw_outstats.loc[:, ['rel_occ','Days']].reset_index().merge(properties_raw_eff[["eff"]], on=['Fuel', 'Type'], how='inner')

properties_stage_outstats['NewType'] = np.where(
     (properties_stage_outstats['Fuel'] == 'Gas') & (properties_stage_outstats['Type'].str.contains('CCGT', regex= True)), 'CCGT', 
        np.where((properties_stage_outstats['Fuel'] == 'Gas') & (~properties_stage_outstats['Type'].str.contains('CCGT', regex= True)), 'OCGT', 
        'rel')
)

## Load generators

In [594]:
power_plants = pm.powerplants()

In [596]:
power_plants['NewTechnology'] = np.where(
     (power_plants['Fueltype'] == 'Natural Gas') & 
        (power_plants['Technology'].str.contains('CCGT', case=False)), 'CCGT', 
        np.where((power_plants['Fueltype'] == 'Natural Gas') & 
        (~power_plants['Technology'].fillna('rel').str.contains('CCGT', case=False)), 'OCGT', 'rel')
)

## Add outage parameters to generators

### Add default values

In [597]:
power_plants[["unavail_rate_def"]] = properties_stage_outstats.loc[:, ['rel_occ']].mean()
power_plants[["unavail_days_def"]] = properties_stage_outstats.loc[:, ['Days']].mean()

### Add more precise values for fuel and type

In [598]:
dict_fuel = {'Hard coal': 'Hard Coal',
'Hydrogen': 'Hydro',
'Lignite': 'Lignite',
'Gas': 'Natural Gas',
'Nuclear': 'Nuclear',
'Heavy oil': 'Oil',
'Light oil': 'Oil',
'Oil shale': 'Oil'}

In [599]:
properties_stage_outstats["pp_fuel"] = properties_stage_outstats['Fuel'].map(dict_fuel)

In [610]:
power_plants["unavail_days_fueltype"] = power_plants.set_index(
    ['Fueltype','NewTechnology']).index.map(
        dict(properties_stage_outstats.groupby(["pp_fuel", "NewType"]).Days.mean()))
power_plants["unavail_rate_fueltype"] = power_plants.set_index(
    ['Fueltype','NewTechnology']).index.map(
        dict(properties_stage_outstats.groupby(["pp_fuel", "NewType"]).rel_occ.mean()))

### Add more precise values based on generator efficiency

In [ ]:
# For selected fuels only the rate varies. The unavailable number of days is 1 for all.
s = power_plants.loc[
    power_plants['Fueltype'].isin(['Hard Coal', 'Lignite'])].Efficiency.transform(
        lambda x: pd.cut(x, bins=[0, 0.43999, 1], labels=[0.1, 0.075])).astype(float)
power_plants.loc[s.index, 'unavail_rate_eff'] = s

s = power_plants.loc[
    power_plants['Fueltype'].isin(['Natural Gas']) & power_plants['NewTechnology'].isin(['CCGT'])].Efficiency.transform(
        lambda x: pd.cut(x, bins=[0, 0.52999, 1], labels=[0.05, 0.08])).astype(float)
power_plants.loc[s.index, 'unavail_rate_eff'] = s

s = power_plants.loc[
    power_plants['Fueltype'].isin(['Natural Gas']) & power_plants['NewTechnology'].isin(['OCGT'])].Efficiency.transform(
        lambda x: pd.cut(x, bins=[0, 0.38999, 1], labels=[0.05, 0.08])).astype(float)
power_plants.loc[s.index, 'unavail_rate_eff'] = s

### Consolidate final parameters

In [612]:
power_plants['unavail_rate'] = np.where(~power_plants.unavail_rate_eff.isna(), power_plants.unavail_rate_eff, 
    np.where(~power_plants.unavail_rate_fueltype.isna(), power_plants.unavail_rate_fueltype, power_plants.unavail_rate_def))

power_plants['unavail_days'] = np.where(~power_plants.unavail_days_fueltype.isna(), power_plants.unavail_days_fueltype, power_plants.unavail_days_def)

In [613]:
power_plants.groupby(['Fueltype', 'NewTechnology'])['unavail_rate'].agg(['unique'])

unique
Fueltype    NewTechnology                                   
Bioenergy   rel                         [0.0751923076923077]
Geothermal  rel                         [0.0751923076923077]
Hard Coal   rel                         [0.0875, 0.075, 0.1]
Hydro       rel                                       [0.05]
Lignite     rel                                [0.0875, 0.1]
Natural Gas CCGT                          [0.06, 0.05, 0.08]
            OCGT           [0.07250000000000001, 0.05, 0.08]
Nuclear     rel                                       [0.05]
Oil         rel                                      [0.091]
Other       rel                         [0.0751923076923077]
Solar       rel                         [0.0751923076923077]
Waste       rel                         [0.0751923076923077]
Wind        rel                         [0.0751923076923077]

In [614]:
power_plants.groupby(['Fueltype', 'NewTechnology'])['unavail_days'].agg(['unique'])

unique
Fueltype    NewTechnology                      
Bioenergy   rel            [1.2307692307692308]
Geothermal  rel            [1.2307692307692308]
Hard Coal   rel                           [1.0]
Hydro       rel                           [1.0]
Lignite     rel                           [1.0]
Natural Gas CCGT                          [1.0]
            OCGT                          [1.0]
Nuclear     rel                           [7.0]
Oil         rel                           [1.0]
Other       rel            [1.2307692307692308]
Solar       rel            [1.2307692307692308]
Waste       rel            [1.2307692307692308]
Wind        rel            [1.2307692307692308]

## Simulate forced outages

In [ ]:
year = 2030
hours_per_year = int((datetime.datetime(year, 12, 31) - datetime.datetime(year, 1, 1)).total_seconds()/3600)

In [647]:
# Inputs: hours_per_year, generator dataframe

generators = power_plants.index
pp = power_plants

np_forcedoutages = np.zeros((len(power_plants), hours_per_year), dtype=int)

for i, gen in enumerate(generators):
    rate = pp.unavail_rate[gen]
    num_drawn_outages = int(365 * rate * 2)
    mean_interval = 24/rate # days_per_year * 24 / outages_per_year = days_per_year * 24 / (days_per_year * pp.unavail_rate[gen])
    duration = pp.unavail_days[gen] * 24 # convert to hours
    
    # Generate inter-arrival times (in hours) using exponential distribution
    times = np.cumsum(np.random.exponential(mean_interval, num_drawn_outages))

    # Only keep outage start times within the year
    outage_starts = times[times < hours_per_year].astype(int)

    for start in outage_starts:
        end = int(min(start + duration, hours_per_year))
        np_forcedoutages[i, start:end] = 1

In [648]:
pdf_forcedoutages = pd.DataFrame(np_forcedoutages, index=pp.index)

In [652]:
pdf_forcedoutages.to_hdf("/trinity/home/fuhrand/ERAA/open_eraa/data/temp/forced_outages.hdf", "forced_outages")